In [2]:
import csv
import pandas as pd
from itertools import combinations
import time


#Read data from CSV
# data = pd.read_csv('groceries.csv')
data = pd.read_csv('Market_Basket_Optimisation.csv', header=None)
#
#Parameters
minsup = float(input("Support-Threshold: "))
minsup = minsup * len(data)
minconf = float(input("Confidence-Threshold: "))
print(len(data))#9835
time_start = time.time()
#Add all data in a list of lists
items = []  
# print(len(data.values[0]))#1列(ROW)有32的物品
for i in range(0, len(data)):  #將所有data中的物品一列一列放到items
     items.append([str(data.values[i, j]) for j in range(0, len(data.values[0]))])

#Creating a list of dictionaries
count = [dict() for x in range(len(data.values[0]) + 1)]
# print(items)
# #Count support for each individual items
s=[]
for i in items:
    for j in i:      #讀取所有items
        s.append(j)
for i in s:                      #創字典 放在count[1]
    #If item is present in dictionary, increment its count by 1
    if i in count[1]:
        count[1][i] = count[1][i] + 1
    #If item is not present in dictionary, set its count to 1
    else:
        count[1][i] = 1     #{'i':1}

# #Remove infrequent and empty items #去除小於minsup,nan
for i in count[1].copy():
    if(count[1][i] < minsup):
        count[1].pop(i)
count[1].pop('nan') #去除以key = 'NAN'產生的值

# #Storing transactions as lists without infrequent items
a = list(count[1]) #列出所有符合的KEY
item = [list() for i in range(len(data))] #創造len(data) = 9835個list 
c = 0 
for i in range(0,len(items)): ##將所有data中的物品一列一列放到items
    for j in range(len(items[i])):
#         print(items[i][j])
        if(a.__contains__(items[i][j]) != 0): #確認物品是否存在
            item[i].append(items[i][j]) #將物品分成[['a', 'b', 'c'],[]]與itmes不同的地方是裡練已經沒有NAN了

#Function to sort list to support
def sort(a): #a = 所有符合的KEY的list
    for i in range(len(a) - 1):
        for j in range(len(a) - i - 1):
            if(count[1][a[j]] < count[1][a[j + 1]]): #如果下一個比上一個大則互換位子
                a[j],a[j + 1] = a[j + 1],a[j]
                
#Call function to sort all transactions in descending order of their support
for i in range(0,len(data)): #9835
    if(len(item[i]) > 1): #如果不只一個物件就排序他
        sort(item[i])

#Tree class for FP-Tree
class tree: #設計樹根，分支
    def __init__(self, name, sup, parent):
        self.name = name
        self.sup = sup
        self.nodeLink = None
        self.parent = parent
        self.children = []

#Function to check if the node is present is a child of the current node
def ispresent(node,name):    #確認是否有children
    f = -1               
    for i in node.children:
        f += 1
        if(i.name == name):
            return f
    return -1

#HeaderTable which stores the reference of last/first occurence of an item. Used as a linked list to generate candidate trees 
lastocc = count[1].copy() #符合sup的字典
for i in lastocc: #lastocc字典中所有值皆為NONE
    lastocc[i] = None
#Function to create FP-tree
root = tree("root", -1, None)
z = 0
for i in item: #item為沒有NAN的items
    current = root
    for j in range(len(i)): #len(i)9835
        if(ispresent(current,i[j])>=0): #第一次通常不符合此情形，直接執行else，看有沒有children
            current = current.children[ispresent(current, i[j])]
            current.sup=current.sup + 1
        else:
            child=tree(i[j], 1, current)    #創建i[j]子根
            current.children.append(child) #將子根放入children =[] 
            t = current #t 記錄柱上一層的tree1
            current = current.children[ispresent(current, i[j])] #現在current.children裡有子根，所以self.name = i[j]如下示意，
            #ispresent(current,i[j])回傳 0 ，現在的current為tree2
            current.parent = t #紀錄上一層tree1
            if(lastocc[current.name] == None):
                lastocc[current.name] = current #創建{'i[j]': tree2-1()} 
            else:#創出第一層並列樹枝
                current.nodeLink = lastocc[current.name] #將tree2-2連結到tree4
                lastocc[current.name] = current #CURRENT = TREE2-2 {'i[j]': tree2-2}
            
# class tree1:
#     def __init__(self, name, sup, parent):
#         self.name = 'root'
#         self.sup = -1
#         self.nodeLink = None
#         self.parent = none
#         self.children = [
#   class tree2-1:
#     def __init__(self, name, sup, parent):      [['A','B','C'],['C','A']]
#         self.name = 'A'
#         self.sup = 1
#         self.nodeLink = NONE
#         self.parent = parent
#         self.children = [
#                                     class tree3:
#                                         def __init__(self, name, sup, parent):
#                                             self.name = 'B'
#                                             self.sup = 1
#                                             self.nodeLink = None
#                                             self.parent = parent
#                                             self.children = [
#                                                                 class tree4:
#                                                                     def __init__(self, name, sup, parent):
#                                                                         self.name = 'B'
#                                                                         self.sup = 1
#                                                                         self.nodeLink = None
#                                                                         self.parent = parent
#                                                                         self.children = []
#                                                                 ]
#         ],
#   class tree2-2:
#     def __init__(self, name, sup, parent):
#         self.name = 'C'
#         self.sup = sup
#         self.nodeLink = TREE 4
#         self.parent = TREE1
#         self.children = []
# ]

#Function to extract single item set from a tuple
def value(a):
    a = str(a)
    a = a[:-2]
    a = a[2:]
    a = a[:-1]
    return a

#Function to get frequent itemsets with suffix 'node' and length n
def singlepath(node, n):
    c = 0
    sup = node.sup
    path = []
    pathname = []
    current = node
    
    #Get the path from current node to root
    while(current.parent!=None): #查看目current是否為'root'，如果是則無parent，重複做直道回到最上層('root')
        path.append(current)     #將current內容紀錄(tree)
        pathname.append(current.name)      #('c')
        current = current.parent  #往上一層移動
    path.remove(node) #除了自己以外的路線
    pathname.remove(node.name) 
    candidatepath = []
    temp_candidatepath = []
   
    #Generate combinations of length n in the path
    a = (list(combinations(pathname, n))) #列出除了自己以外的長度為n的組合
    for j in a:
        temp_candidatepath.append(tuple(sorted(j)))   #以tuple形式存在temp_candidatapath裡
    #Append the suffix 'node.name' to the above paths
    for j in temp_candidatepath:
        j = list(j) #再次變回list
        j.append(node.name) #將原本剔除的當前TREE名加回去 變成包含當前的組合，就是在做freqent pattern
        candidatepath.append(sorted(j)) #在加回去變成 [['a','c'],['b','c']]
    #Update counts of the generated itemsets
    for j in candidatepath:
        j = tuple(j) #j = ('a','c')
        if j in count[n+1]: #count[1]為符合minsup的字典，n至少會從1開始 
            count[n + 1][j] = count[n+1][j] + sup  # 將之前已創好的value值，加上當前tree所有的sup
        else:
            count[n + 1][j] = sup #創建新字典,此sup為當前tree存有的sup
            
    #Iterating in the candidate tree recursively 
    if(node.nodeLink != None): #查看是否存在分支
        node=node.nodeLink
        singlepath(node, i) #i為組合長度
    
#Check if itemset is frequent
def frequent(n): #算出現次數
    f=0
    for i in count[n]:
        if(count[n][i] >= minsup):
            f=1
    if(f == 1):
        return 1
    else:
        return 0

#Call singlepath function for all frequent nodes
for i in range(1, len(data.values[0]) + 1): #1列(ROW)有32的物品, +1是確保有32為一組的組合  #!
    if(frequent(i) == 1):
        for j in lastocc: #開始製造組合，將結果放到count[]裡，count唯有32空字典的list
            singlepath(lastocc[j], i)

#Remove infrequent itemsets
for z in range(len(data.values[0]) + 1):            
    for i in count[z].copy():
            if(count[z][i] < minsup): #查看所有的dict裡面的東西，是否符合minsup
                count[z].pop(i) #去掉不符合的
frequent_pattern = []
for items in count:
    for want in items:
        frequent_pattern.append(want)
                
#Get 'q', the length of the longest itemset
i=2
while(len(count[i-1]) != 0): #從這邊就能知道最大符合長度是多長，因為不符合的都被刪掉了
    i = i + 1
q = i - 2

#Find maximal and closed itemsets
maximal = []
closed = []
for i in range(1, q):
    for j in count[i]:
        fm = 0
        fc = 0
        for k in count[i+1]:
            a = set(list([j]))
            b = set(list(k))
            #Set is maximal if no immediate superset is frequent
            if(a.intersection(b) == a): #如果兩者重疊則maximal
                fm = 1 
                #Set is closed if none of its immediate supersets have equal support
                if(count[i][j] == count[i+1][k]): #出現次數相同
                    fc = 1
        if(fm == 0):
            maximal.append(j)
        if(fc == 0):
            closed.append(j)
#All sets at the top of the tree are automatically maximal and closed
for i in count[q]:
    maximal.append(i)
    closed.append(i)
print("frequent_pattern")
print(len(frequent_pattern))
print(frequent_pattern)
    
#Find Association Rules 
print("Association_Rules")
ant=count.copy()
for i in range(q, 0, -1): #從後面樹回來
    for j in ant[i]:  #ant = count
        for k in range(i-1, 0, -1): # i = q(最大長度)
            s = list(combinations(list(j), k))  #將所有可能組出來   
            #Traverse through list of all combinations of antecedants
            for n in s:
                #Sorting to prevent duplicate itemsets 
                r=tuple(sorted(set(j).difference(set(n))))
                l=len(n) #輸入進來的字元長度
                #Check if len(n)==1 to be able to extract key to search in the support dictionary. 
                if(l == 1):
                    n=value(n)
                    l=1
                if(len(r) == 1):
                    r2=value(r)
                if(n != None):
                    #If rule's confidence is greater than minconfidence, then print the rule
                    if((ant[len(j)][j] / ant[l][n]) >= minconf):
                        #Rule is only significant if it is present in CLOSED, otherwise it is redundant
                        if(closed.__contains__((n))):
                            c += 1
                            if(len(r) == 1):
                                print(n,"(",ant[l][n],")","--->",r2,"(",ant[len(r)][r2],")", " - conf(",(ant[len(j)][j]/ant[l][n]),")")
                            else:
                                print(n,"(",ant[l][n],")","--->",r,"(",ant[len(r)][r],")", " conf(",(ant[len(j)][j]/ant[l][n]),")")
print(c, "rules generated")
time_end = time.time()
print('FP_Growth_spent_time', time_end - time_start)
# print( time_end - time_start)

Support-Threshold: 0.0045
Confidence-Threshold: 0.1
7501
frequent_pattern
843
['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes', 'whole weat flour', 'yams', 'cottage cheese', 'energy drink', 'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad', 'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie', 'spinach', 'olive oil', 'burgers', 'meatballs', 'eggs', 'turkey', 'milk', 'energy bar', 'whole wheat rice', 'whole wheat pasta', 'french fries', 'soup', 'light cream', 'shallot', 'frozen vegetables', 'spaghetti', 'pet food', 'cookies', 'cooking oil', 'champagne', 'chocolate', 'chicken', 'oil', 'fresh tuna', 'tomatoes', 'black tea', 'extra dark chocolate', 'protein bar', 'red wine', 'pasta', 'pepper', 'shampoo', 'rice', 'sparkling water', 'ham', 'body spray', 'pancakes', 'grated cheese', 'white wine', 'toothpaste', 'parmesan cheese', 'fresh bread', 'ground beef', 'escalope', 'herb & pepper', 'tomato sauce', 'magazines', 'strawberries', 'strong cheese', '

('burgers', 'milk') ( 134 ) ---> chocolate ( 1230 )  - conf( 0.30597014925373134 )
('chocolate', 'milk') ( 241 ) ---> burgers ( 654 )  - conf( 0.17012448132780084 )
('burgers', 'milk') ( 134 ) ---> spaghetti ( 1306 )  - conf( 0.3582089552238806 )
('burgers', 'spaghetti') ( 161 ) ---> milk ( 972 )  - conf( 0.2981366459627329 )
('milk', 'spaghetti') ( 266 ) ---> burgers ( 654 )  - conf( 0.18045112781954886 )
('burgers', 'milk') ( 134 ) ---> mineral water ( 1788 )  - conf( 0.3880597014925373 )
('burgers', 'mineral water') ( 183 ) ---> milk ( 972 )  - conf( 0.28415300546448086 )
('milk', 'mineral water') ( 360 ) ---> burgers ( 654 )  - conf( 0.14444444444444443 )
('burgers', 'ground beef') ( 90 ) ---> mineral water ( 1788 )  - conf( 0.4444444444444444 )
('burgers', 'mineral water') ( 183 ) ---> ground beef ( 737 )  - conf( 0.2185792349726776 )
('ground beef', 'mineral water') ( 307 ) ---> burgers ( 654 )  - conf( 0.13029315960912052 )
('burgers', 'french fries') ( 165 ) ---> green tea ( 99

('cooking oil', 'spaghetti') ( 119 ) ---> mineral water ( 1788 )  - conf( 0.4789915966386555 )
('mineral water', 'spaghetti') ( 448 ) ---> cooking oil ( 383 )  - conf( 0.12723214285714285 )
cooking oil ( 383 ) ---> ('mineral water', 'spaghetti') ( 448 )  conf( 0.14882506527415143 )
('cooking oil', 'mineral water') ( 151 ) ---> pancakes ( 713 )  - conf( 0.23178807947019867 )
('cooking oil', 'pancakes') ( 59 ) ---> mineral water ( 1788 )  - conf( 0.5932203389830508 )
('mineral water', 'pancakes') ( 253 ) ---> cooking oil ( 383 )  - conf( 0.1383399209486166 )
('cooking oil', 'eggs') ( 88 ) ---> mineral water ( 1788 )  - conf( 0.5454545454545454 )
('cooking oil', 'mineral water') ( 151 ) ---> eggs ( 1348 )  - conf( 0.31788079470198677 )
('eggs', 'mineral water') ( 382 ) ---> cooking oil ( 383 )  - conf( 0.1256544502617801 )
cooking oil ( 383 ) ---> ('eggs', 'mineral water') ( 382 )  conf( 0.12532637075718014 )
('chocolate', 'cooking oil') ( 102 ) ---> eggs ( 1348 )  - conf( 0.3333333333333

almonds ( 153 ) ---> chocolate ( 1230 )  - conf( 0.29411764705882354 )
almonds ( 153 ) ---> milk ( 972 )  - conf( 0.2549019607843137 )
almonds ( 153 ) ---> mineral water ( 1788 )  - conf( 0.37254901960784315 )
almonds ( 153 ) ---> burgers ( 654 )  - conf( 0.2549019607843137 )
avocado ( 250 ) ---> milk ( 972 )  - conf( 0.224 )
avocado ( 250 ) ---> mineral water ( 1788 )  - conf( 0.348 )
avocado ( 250 ) ---> frozen vegetables ( 715 )  - conf( 0.14 )
avocado ( 250 ) ---> green tea ( 991 )  - conf( 0.188 )
avocado ( 250 ) ---> french fries ( 1282 )  - conf( 0.24 )
avocado ( 250 ) ---> burgers ( 654 )  - conf( 0.136 )
avocado ( 250 ) ---> chocolate ( 1230 )  - conf( 0.212 )
avocado ( 250 ) ---> frozen smoothie ( 475 )  - conf( 0.152 )
avocado ( 250 ) ---> spaghetti ( 1306 )  - conf( 0.24 )
avocado ( 250 ) ---> eggs ( 1348 )  - conf( 0.168 )
vegetables mix ( 193 ) ---> chocolate ( 1230 )  - conf( 0.18652849740932642 )
vegetables mix ( 193 ) ---> eggs ( 1348 )  - conf( 0.18652849740932642 )
v

frozen vegetables ( 715 ) ---> eggs ( 1348 )  - conf( 0.22797202797202798 )
french fries ( 1282 ) ---> frozen vegetables ( 715 )  - conf( 0.11154446177847113 )
frozen vegetables ( 715 ) ---> french fries ( 1282 )  - conf( 0.2 )
frozen vegetables ( 715 ) ---> ground beef ( 737 )  - conf( 0.17762237762237762 )
ground beef ( 737 ) ---> frozen vegetables ( 715 )  - conf( 0.1723202170963365 )
frozen vegetables ( 715 ) ---> spaghetti ( 1306 )  - conf( 0.2923076923076923 )
spaghetti ( 1306 ) ---> frozen vegetables ( 715 )  - conf( 0.16003062787136293 )
frozen vegetables ( 715 ) ---> mineral water ( 1788 )  - conf( 0.3748251748251748 )
mineral water ( 1788 ) ---> frozen vegetables ( 715 )  - conf( 0.14988814317673377 )
eggs ( 1348 ) ---> spaghetti ( 1306 )  - conf( 0.2032640949554896 )
spaghetti ( 1306 ) ---> eggs ( 1348 )  - conf( 0.20980091883614088 )
mineral water ( 1788 ) ---> spaghetti ( 1306 )  - conf( 0.2505592841163311 )
spaghetti ( 1306 ) ---> mineral water ( 1788 )  - conf( 0.3430321